In [ ]:
from crossing import preprop_GMTs, cross_GMTs
import pandas as pd
import time
from tqdm import tqdm 
import os
import numpy as np

In [ ]:
preprop_GMTs("data/latest.gmt", "data/rummagene")
print("rummagene.csv created")
preprop_GMTs("data/human-geo-auto.gmt.gz", "data/rummageo_human")
print("rummageo_human.csv created")
preprop_GMTs("data/mouse-geo-auto.gmt.gz", "data/rummageo_mouse")
print("rummageo_mouse.csv created")


In [ ]:
rummagene = pd.read_csv("data/rummagene.csv")
rummageo_h = pd.read_csv("data/rummageo_human.csv")
rummageo_m = pd.read_csv("data/rummageo_mouse.csv")

In [ ]:
def rummagene_exists(df, file_path, column="identifier"):
    "takes df and removes from its given column terms already present in file_path to avoid running crossing that is already done"
    if not os.path.isfile(file_path):
        open(file_path, 'w').close()  
    
    with open(file_path, 'r') as file:
        existing_elements = {line.strip() for line in file}
    
    filtered_df = df[~df[column].isin(existing_elements)]
    return filtered_df

In [ ]:
def cross_files(rumma_geo, output_prefix, record_file):
    batch_size = 5000
    num_batches = (len(rummagene) // batch_size) + 1


    for i in range(num_batches):
        start = time.time()
        start_idx = i * batch_size
        end_idx = start_idx + batch_size
        batch_n = rummagene[start_idx:end_idx]
        main_dict = {} #maps rummagene name to rummagene genes
        desc_dict ={} #maps rummagene name to rummagene table description
        batch = rummagene_exists(batch_n, record_file)
        if len(batch) != 0:
            print(f"This is {len(batch)} rows")
            for index, row in tqdm(batch.iterrows(), total=batch.shape[0]):
                main_dict[row["identifier"]] = set(row['genes'].split(";"))
                desc_dict[row['identifier']] = row["desc"]
            if  len(main_dict) > 0:
                rummageo_dict = {row_inner["identifier"]: set(row_inner["genes"].split(";")) for index_inner, row_inner in rumma_geo.iterrows()}
                cross_GMTs(main_dict,desc_dict, rummageo_dict, output_prefix, record_file)

        else:
            print("O batch")
        elapsed_time = time.time() - start
        print(f"batch_done:  {int(elapsed_time // 3600)}hr, {int((elapsed_time % 3600) // 60)}min, {int(elapsed_time % 60)}sec")
        elapsed_time = time.time() - start
        print(f"full_done:  {int(elapsed_time // 3600)}hr, {int((elapsed_time % 3600) // 60)}min, {int(elapsed_time % 60)}sec")
        print(i)
cross_files(rummageo_h,"data/rummageogene_human", "data/rummagene_h.txt")
cross_files(rummageo_m,"data/rummageogene_mouse", "data/rummagene_m.txt")

In [ ]:
human = pd.read_csv("data/rummageogene_human.csv")
human["species"] = "human"
mouse = pd.read_csv("data/rummageogene_mouse.csv")
mouse["species"] = "mouse"
df = pd.concat([human, mouse])
df = df.drop_duplicates(subset=["rummagene", "rummageo"])
df = df.sort_values(by=["p-value", "odds"], ascending=[True, False])
df.index = np.arange(1, len(df.index)+1)
df

In [ ]:
df.to_csv('data/rummagenexrummageo.csv', index=False)
